In [ ]:
import matplotlib.pyplot as plt
import mplhep as hep
import ROOT as R
from triggercalib import HltEff
import uproot

hep.style.use("LHCb2")
R.EnableImplicitMT(8)

In [ ]:
hlt_eff = HltEff(
    "bu2jpsik",
    "root://eoslhcb.cern.ch//eos/lhcb/wg/rta/WP4/TriggerCalib/example_bu2jpsik_magdown_mc_example.root:Tuple/DecayTree",
    probe="Hlt1TrackMVA",
    tag=["Hlt1TrackMVA", "Hlt1TwoTrackMVA"],
    particle="B",
    binning={
        "B_PT" : {
            "bins" : [
                n*1e3 for n in (
                    2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,
                    14, 15, 16, 17, 18, 19, 20, 21, 22, 25
                )
            ]
        }
    },
    sideband={
        "B_DTF_PV_Jpsi_mass_const_M": {
            "range": [5280 - 150, 5280 + 150], # Define outer sideband edges as 5130 and 5430
            "sideband": [5280 - 55, 5280 + 55], # Define inner sideband edges as 5225 and 5335 (signal window inferred between)
        }
    }
)

In [ ]:
tos_eff_hist = uproot.from_pyroot(hlt_eff["efficiencies"]["tos_efficiency_B_PT"])

# Prepare quantities for plot#
values, bins = tos_eff_hist.to_numpy()
errors = tos_eff_hist.errors()
midpoints = (bins[1:] + bins[:-1]) / 2 / 1e3
widths = (bins[1:] - bins[:-1]) / 2 / 1e3

xmin = midpoints[0] - widths[0]
xmax = midpoints[-1] + widths[-1]

# Plot efficiency #
plt.figure(figsize=(12,10))

plt.plot((xmin, xmax), (1,1), color='k', ls='dashed', lw=2)
plt.errorbar(
    x=midpoints, y=values, xerr=widths, yerr=errors,
    color='r', elinewidth=2, ls="none", marker='.', markersize=8, 
)

plt.xlim(xmin, xmax)
plt.xlabel(r"Transverse momentum, $p_T\left(B^+\right)$ / $\mathrm{GeV}c^{-2}$")

plt.ylim(0, 1.1)
plt.ylabel(r"TOS efficiency, $\varepsilon_\mathrm{TOS}$")

hep.lhcb.label(loc=0, rlabel=r"$B^+\to J/\psi\left(\mu\mu\right)K^+$ 2024 MC")

plt.show()